In [ ]:
#| default_exp 10_msmarco-generated-entities

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| export
import json, re, scipy.sparse as sp, pickle, numpy as np, argparse, pandas as pd

from tqdm.auto import tqdm
from dataclasses import dataclass

from sugar.core import *

## Load entity

In [ ]:
#| export
def extract_key(entities, key='entity_canonical_category'):
    if key == 'entity_canonical_category': 
        return [o.lower() for o in entities if len(o.split(' | ')) == 3]
    elif key == 'entity': 
        return [o.split(' | ')[0].lower() for o in entities if len(o.split(' | ')) == 3]
    elif key == 'canonical': 
        return [o.split(' | ')[1].lower() for o in entities if len(o.split(' | ')) == 3]
    elif key == 'category': 
        return [o.split(' | ')[2].lower() for o in entities if len(o.split(' | ')) == 3]
    elif key == 'entity_canonical': 
        return [' | '.join(o.split(' | ')[0:2]).lower() for o in entities if len(o.split(' | ')) == 3]
    else: 
        raise ValueError(f'Invalid entity type: {key}')
    

In [ ]:
#| export
def load_gpt_entities(fname, key='entity_canonical_category'):
    with open(fname, 'rb') as file:
        generations = pickle.load(file)
        
    entities = dict()
    for k,v in tqdm(generations.items(), total=len(generations)):
        query = k.split('\t')[1].split('==::==')[0].split('|||')[1]
        entities.setdefault(query, []).extend(extract_key(v['Entities'], key))
        
    return entities


In [ ]:
#| export
def load_llama_entities(fname, key='entity_canonical_category'):
    with open(fname, 'r') as file:
        generations = [json.loads(line) for line in file]

    entities = dict()
    pattern = r"<Entities>(.*?)</Entities>"
    for generation in tqdm(generations, total=len(generations)):
        entity = [entity.strip() for entity in re.findall(pattern, generation['gen'], re.DOTALL) if len(entity.strip().split(' | ')) == 3]
        entities.setdefault(int(generation['qid']), []).extend(extract_key(entity, key))
        
    return entities
    

In [ ]:
entity_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/entities/gpt/"

trn_entity = load_gpt_entities(f'{entity_dir}/trn_gpt_generations.pkl', key='entity')
tst_entity = load_gpt_entities(f'{entity_dir}/dev_gpt_generations.pkl', key='entity')

  0%|          | 0/532743 [00:00<?, ?it/s]

  0%|          | 0/34621 [00:00<?, ?it/s]

In [ ]:
print(list(trn_entity)[:3])

['what is eq in sound', 'when was-king tut exhibit in denver', 'how to put a roof on your deck']


In [ ]:
entity_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/entities/llama/"

trn_entity = load_llama_entities(f'{entity_dir}/trn_llama_generations.txt', key='entity')
tst_entity = load_llama_entities(f'{entity_dir}/dev_llama_generations.txt', key='entity')

  0%|          | 0/532755 [00:00<?, ?it/s]

  0%|          | 0/59265 [00:00<?, ?it/s]

In [ ]:
print(list(trn_entity)[:3])

[1185869, 1185868, 597651]

In [ ]:
#| export
def load_entities(entity_dir, key='entity_canonical_category', model='gpt'):
    mapping = dict()
    if model == 'gpt': 
        mapping.update(load_gpt_entities(f'{entity_dir}/trn_gpt_generations.pkl', key=key))
        mapping.update(load_gpt_entities(f'{entity_dir}/dev_gpt_generations.pkl', key=key))
    elif model == 'llama': 
        mapping.update(load_llama_entities(f'{entity_dir}/trn_llama_generations.txt', key=key))
        mapping.update(load_llama_entities(f'{entity_dir}/dev_llama_generations.txt', key=key))
    else: 
        raise ValueError(f'Invalid pretrained type: {model}')

    return mapping


## Create matrix

In [ ]:
#| export
def _get_all_ids(data_dir):
    trn_ids, trn_txt = load_raw_file(f'{data_dir}/raw_data/train.raw.txt')
    tst_ids, tst_txt = load_raw_file(f'{data_dir}/raw_data/test.raw.txt')
    lbl_ids, lbl_txt = load_raw_file(f'{data_dir}/raw_data/label.raw.txt')
    trn_ids, tst_ids, lbl_ids = list(map(int, trn_ids)), list(map(int, tst_ids)), list(map(int, lbl_ids))
    return trn_ids, trn_txt, tst_ids, tst_txt, lbl_ids, lbl_txt
    

In [ ]:
#| export
def get_matrix(mapping, trn_ids, trn_txt, tst_ids, tst_txt, vocab_size, model='gpt'):
    if model == 'gpt':
        trn_mat, _ = get_matrix_from_item2idx(mapping, vocab_size, ids=trn_txt)
        tst_mat, _ = get_matrix_from_item2idx(mapping, vocab_size, ids=tst_txt)
    elif model == 'llama':
        trn_mat, _ = get_matrix_from_item2idx(mapping, vocab_size, ids=trn_ids)
        tst_mat, _ = get_matrix_from_item2idx(mapping, vocab_size, ids=tst_ids)
    else:
        raise ValueError(f'Invalid model type: {model}.')

    return trn_mat, tst_mat
    

In [ ]:
#| export
def filter_vocab(vocab_ids, vocab_txt, trn_mat, tst_mat, lbl_mat=None):
    valid_idx = np.where(trn_mat.getnnz(axis=0) > 0)[0]
    if lbl_mat is not None:
        lbl_idx = np.where(lbl_mat.getnnz(axis=0) > 0)[0]
        valid_idx = np.union1d(valid_idx, lbl_idx)

    trn_mat = trn_mat[:, valid_idx].copy()
    tst_mat = tst_mat[:, valid_idx].copy()
    if lbl_mat is not None: 
        lbl_mat = lbl_mat[:, valid_idx].copy()
    
    vocab_ids, vocab_txt = [vocab_ids[i] for i in valid_idx], [vocab_txt[i] for i in valid_idx]

    return vocab_ids, vocab_txt, trn_mat, tst_mat, lbl_mat
    

In [ ]:
#| export 
def get_entities(data_dir:str, entity_dir:str, key='entity_canonical_category', model='gpt'):
    mapping = load_entities(entity_dir, key, model)

    trn_ids, trn_txt, tst_ids, tst_txt, lbl_ids, lbl_txt = _get_all_ids(data_dir)

    vocab, mapping_item2idx = create_vocab_and_item2idx(mapping)
    entity_ids, entity_txt = list(range(len(vocab))), list(sorted(vocab, key=lambda x: vocab[x]))

    trn_mat, tst_mat = get_matrix(mapping_item2idx, trn_ids, trn_txt, tst_ids, tst_txt, len(entity_ids), model=model)
    lbl_mat = sp.csr_matrix((len(lbl_ids), len(entity_ids)), dtype=np.int64)

    entity_ids, entity_txt, trn_mat, tst_mat, lbl_mat = filter_vocab(entity_ids, entity_txt, trn_mat, tst_mat, lbl_mat)
    return trn_mat, tst_mat, lbl_mat, entity_ids, entity_txt
    

In [ ]:
#| export
def save_entities(save_dir, trn_mat, tst_mat, lbl_mat, entity_ids, entity_txt, key, model):
    sp.save_npz(f'{save_dir}/{key}_trn_X_Y_{model}.npz', trn_mat)
    sp.save_npz(f'{save_dir}/{key}_tst_X_Y_{model}.npz', tst_mat)
    sp.save_npz(f'{save_dir}/{key}_lbl_X_Y_{model}.npz', lbl_mat)

    save_raw_txt(f'{save_dir}/raw_data/{key}.{model}.raw.txt', entity_ids, entity_txt)
    

In [ ]:
#| export
def get_and_save_entities(data_dir:str, entity_dir:str, key='entity_canonical_category', model='gpt'):    
    trn_mat, tst_mat, lbl_mat, entity_ids, entity_txt = get_entities(data_dir, entity_dir, key, model=model)
    save_entities(data_dir, trn_mat, tst_mat, lbl_mat, entity_ids, entity_txt, key, model=model)
    return trn_mat, tst_mat, lbl_mat, entity_ids, entity_txt
    

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--entity_dir', type=str, required=True)
    parser.add_argument('--entity_type', type=str, required=True)
    parser.add_argument('--model', type=str, required=True)
    return parser.parse_args()
    

In [ ]:
#| export
if __name__ == '__main__':
    args = parse_args()
    get_and_save_entities(args.data_dir, args.entity_dir, key=args.entity_type, model=args.model)
    

In [ ]:
data_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/XC"
entity_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/entities/gpt/"
key, model = 'entity', 'gpt'

o = get_and_save_entities(data_dir, entity_dir, key=key, model=model)

  0%|          | 0/532743 [00:00<?, ?it/s]

  0%|          | 0/34621 [00:00<?, ?it/s]

  0%|          | 0/535496 [00:00<?, ?it/s]

  0%|          | 0/502939 [00:00<?, ?it/s]

  0%|          | 0/6980 [00:00<?, ?it/s]

In [ ]:
data_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/XC"
entity_dir = "/home/scai/phd/aiz218323/scratch/datasets/msmarco/entities/llama/"
key, model = 'entity', 'llama'

o = get_and_save_entities(data_dir, entity_dir, key=key, model=model)

  0%|          | 0/532755 [00:00<?, ?it/s]

  0%|          | 0/59265 [00:00<?, ?it/s]

  0%|          | 0/558503 [00:00<?, ?it/s]

  0%|          | 0/502939 [00:00<?, ?it/s]

  0%|          | 0/6980 [00:00<?, ?it/s]